In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download aryanrishilamba/hand-gesture-cricket

Dataset URL: https://www.kaggle.com/datasets/aryanrishilamba/hand-gesture-cricket
License(s): CC0-1.0
 97% 1.12G/1.16G [00:11<00:00, 145MB/s]
100% 1.16G/1.16G [00:11<00:00, 108MB/s]


In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompati

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/hand-gesture-cricket.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

def crop_hand(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]

        # Get bounding box from landmarks
        h, w, _ = image.shape
        x_coords = [lm.x * w for lm in hand_landmarks.landmark]
        y_coords = [lm.y * h for lm in hand_landmarks.landmark]

        x_min, x_max = int(min(x_coords)), int(max(x_coords))
        y_min, y_max = int(min(y_coords)), int(max(y_coords))

        # Add padding
        padding = 20
        x_min = max(0, x_min - padding)
        y_min = max(0, y_min - padding)
        x_max = min(w, x_max + padding)
        y_max = min(h, y_max + padding)

        cropped = image[y_min:y_max, x_min:x_max]
        return cropped
    return None


In [ ]:
import os

input_dir = '/content/My_Hand_Gesture_Dataset/My_Hand_Gesture_Dataset/Testing_Data'
output_dir = '/content/Testing'

os.makedirs(output_dir, exist_ok=True)

for label in os.listdir(input_dir):
    label_path = os.path.join(input_dir, label)
    output_label_path = os.path.join(output_dir, label)
    os.makedirs(output_label_path, exist_ok=True)

    for img_file in os.listdir(label_path):
        img_path = os.path.join(label_path, img_file)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Skipping unreadable file: {img_path}")
            continue

        cropped = crop_hand(img)
        if cropped is not None:
            save_path = os.path.join(output_label_path, img_file)
            cv2.imwrite(save_path, cropped)


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from tqdm import tqdm

# Setup Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)
input_dir = "/content/Training"
output_dir = "dataset_processed"
os.makedirs(output_dir, exist_ok=True)

def preprocess_and_save(image, save_path):
    h, w, _ = image.shape
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)

    if result.multi_hand_landmarks:
        lm = result.multi_hand_landmarks[0]
        x_coords = [int(lmpt.x * w) for lmpt in lm.landmark]
        y_coords = [int(lmpt.y * h) for lmpt in lm.landmark]
        x_min = max(min(x_coords) - 30, 0)
        y_min = max(min(y_coords) - 30, 0)
        x_max = min(max(x_coords) + 30, w)
        y_max = min(max(y_coords) + 30, h)

        hand_roi = image[y_min:y_max, x_min:x_max]

        # Skin segmentation in HSV
        hsv = cv2.cvtColor(hand_roi, cv2.COLOR_BGR2HSV)
        lower_skin = np.array([0, 30, 60], dtype=np.uint8)
        upper_skin = np.array([20, 150, 255], dtype=np.uint8)
        mask = cv2.inRange(hsv, lower_skin, upper_skin)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask = cv2.erode(mask, kernel, iterations=1)
        mask = cv2.dilate(mask, kernel, iterations=1)
        mask = cv2.GaussianBlur(mask, (7, 7), 0)

        segmented = cv2.bitwise_and(hand_roi, hand_roi, mask=mask)
        gray = cv2.cvtColor(segmented, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (512, 512))

        cv2.imwrite(save_path, resized)
        return True
    return False

# Loop through dataset
for class_name in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    output_class_path = os.path.join(output_dir, class_name)
    os.makedirs(output_class_path, exist_ok=True)

    for img_name in tqdm(os.listdir(class_path), desc=f"Processing class {class_name}"):
        img_path = os.path.join(class_path, img_name)
        save_path = os.path.join(output_class_path, img_name)

        img = cv2.imread(img_path)
        if img is None:
            continue

        success = preprocess_and_save(img, save_path)
        if not success:
            print(f"⚠️ Skipping {img_path} - no hand detected.")


In [ ]:
!pip install split-folders

In [ ]:
import os

dataset_path = "dataset_processed"  # or "dataset" if using original

class_counts = {}

for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_dir):
        continue

    image_files = [f for f in os.listdir(class_dir)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    class_counts[class_name] = len(image_files)

# Print results
for class_name, count in sorted(class_counts.items()):
    print(f"Class {class_name}: {count} images")


Class .ipynb_checkpoints: 0 images
Class 0: 2277 images
Class 1: 3332 images
Class 2: 3335 images
Class 3: 2922 images
Class 4: 3841 images
Class 5: 3997 images
Class 6: 2251 images
Class 7: 3969 images
Class 8: 4000 images
Class 9: 4000 images
Class Nothing: 0 images


In [ ]:
import os
import splitfolders

input_dir = "/content/dataset_processed"
output_dir = 'train_test_split'

os.makedirs(output_dir, exist_ok=True)

input_dir_extracted='/content/Training'

if os.path.isdir(input_dir_extracted):
    splitfolders.ratio(input_dir_extracted, output=output_dir, seed=1337, ratio=(.8, .2), move=False)

Copying files: 39391 files [00:04, 8586.02 files/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    directory='/content/train_test_split/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(512,512),
    color_mode='grayscale'
)
test_ds=tf.keras.utils.image_dataset_from_directory(
    directory='/content/train_test_split/val',
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(512,512),
    color_mode='grayscale'
)

Found 31510 files belonging to 10 classes.
Found 7881 files belonging to 10 classes.


In [ ]:

def process(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label



train_ds=train_ds.map(process)
test_ds=test_ds.map(process)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Lambda
import tensorflow as tf

model = Sequential()


model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='valid', input_shape=(512, 512, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(train_ds,epochs=10,validation_data=test_ds)

In [ ]:
from google.colab import files
model.save("Hand_Numbers_11.h5")
files.download("Hand_Numbers_11.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>